## Rethinking Coaddition Data Products and Tasks
### August 1st 2017

Goals:
 * Come to undertanding of *all the coadds* we'll want to make in a DRP (long term vision). How are they parameterized?
 * Agree on short term solution to RFC that fixes current problems.  

## Current problems

* Tasks that are awkward/impossible
 1. Can't make direct/psfMatched, deep/bestSeeing etc... coadds in one DRP (without --clobber-config!). In LDM-151 these are called DeepCoadd, BestSeeingCoadd and ConstantPsfCoadd. We may want different modelPsfs for different purposes (e.g. deblending, artifact rejection) too.
 2. Implementing artifact rejection in deepCoadds. This requires a PSF-Matched Median coadd as an intermediate data product.
 3. Can't generate series of `ShortPeriodCoadds` without mutliple output repos. 
 4. TemplateCoadds for Alert Production binned by parallactic angle or wavelength to account for DCR.

* The proliferation of data types in obs-base smells bad. 
* The illusion of "deep" being configurable is a lie. It's hard codded the argument parser, for example. 

## Proposal

* Make a new parent Coaddition Task that can make multiple coadd types. This can be subclassed to make more types. 
* Move *some* configs to data ID keys


## Which configs can be data ID keys?

* Driving Principle:
 * _Dataset Types:_ not interchangeblae/substitutable 
 * _Data IDs:_ should be interchangable/substitutable. Keys shouldn't drive the logic. For example, we should not ever have to do the following:

```
if dataId['filter'] == "HSC-Y": 
    # NO! Shouldn't do this
```

* Therefore
 * `WarpType` = data TYPE because downstream measurement on direct/psfMatched/likelihood coadds is different. 
 * `Period` (i.e. `y1`, `y2`, `y3`) = data Id key
 * `ObsCondition` (i.e. `deep`, `bestSeeing`) = data Id key
 * `Subfilter` for DCR = data Id key

### Map values for new keys to configs:

This is analagous to `makeSkyMap.py` which DEFINES the coadd data ID keys: `tract` and `patch`. They're not descriptors of raw data. We invent these keys, and they make to `skyMap` configs.

Therefore, we propose to remove the "deep" specifier and specify the additional coadd keys in 1 or 2 new tasks that:
* `RegisterCoaddsPsfMatchSizes`
* `RegisterCoaddSelection` which maps which config to pass to `ImageSelectionTask`
  * *Epoch criteria*: `Period`
  * *Observing Conditions criteria* such as seeing cutoff (`deep`/`bestSeeing`), transparency cutoff? airmass cutoff?)
 
Notes:
* In the same way that `patch` is a string that maps to a bounding box via the skymap, these tasks will define *string labels* for configs in a per-repo data product analgous to `skyMap.pickle`.
* It defines the how *values* for the keys are defined, but cannot invent new keys unless added to the butler in obs_base. 

## Make new parent "Coaddition Task" (name negotiable)

1. Reimplements coaddition to use an online algorithm (implementation detail). 
2. Can make a series of coadds using keys defined in "registerCoaddSelection"
```
coadditionTask.py input/repo --output output/repo \
--id tract=0 patch=0,0 selection=y1^y2^y3^y4
```